In [ ]:
import os
import json
import numpy as np
import torch
import torch.nn as nn
import cv2
from torch.autograd import Variable

USE_CUDA = torch.cuda.is_available()
DATA_PATH = "../../DATA"
DATA_SET = "test_clip_2017_03_1"
LIST_FILE = "labelled_samples2_done2.json"
TRAIN_SPLIT_RATIO = 0.7
# Model Hyper-parameters
INPUT_CHANNELS = 3
KSIZE1 = 3
KNUM1 = 32
KSIZE2 = 3
KNUM2 = 64
HIDDEN_UNITS_NUM1 = 256
LOCATION_NUM = 120

# DATA

In [ ]:
DETECTION_AREA_WIDTH = 500
DETECTION_AREA_HEIGHT = 20

with open(os.path.join(DATA_PATH, DATA_SET, LIST_FILE), 'r') as f:
    d = json.load(f)
    
roi = d['roi']
samples = d['samples']

In [ ]:

def _get_labelled_images(samples, fullname_fn):
    """
    :param samples a list, samples[i] is
      [filename, [x-coordinates of control points], [y-...]]
    :param fullname_fn: converts file name to full.
    """
    train_images = []
    label_images = []
    mean_bgr_ = np.zeros(3)
    for fname, xs, ys in samples:
        im = cv2.imread(fullname_fn(fname))
        label_im = np.zeros_like(im[:,:,0])
        for x0, x1, y0, y1  in zip(xs[:-1], xs[1:], ys[:-1], ys[1:]):
            cv2.line(label_im, (int(x0), int(y0)), (int(x1), int(y1)), 255, 2)
            
        x0_, x1_ = roi['left'], roi['left'] + roi['width']
        y0_, y1_ = roi['top'], roi['top'] + roi['height']
        
        train_images.append(im[y0_:y1_, x0_:x1_, :])
        label_images.append(label_im[y0_:y1_, x0_:x1_])
        
        mean_bgr_ += train_images[-1].mean(axis=0).mean(axis=0)

        # im2 = cv2.addWeighted(im, 0.5, cv2.cvtColor(label_im, cv2.COLOR_GRAY2BGR), 0.5, 0)
        # cv2.imshow("a", im2)
        # cv2.waitKey()
    # cv2.destroyAllWindows()
    return train_images, label_images, mean_bgr_ / len(train_images)
        
fullname_fn = lambda x: os.path.join(DATA_PATH, x)

     

In [ ]:
def get_label_code_h(label_im, num_classes):
    """
    Encode the horizontal position of the labels -- the left most pixel
    """
    onehot_code = np.zeros(num_classes+1, dtype=np.uint8)
    w = label_im.shape[1]
    _, xs = np.nonzero(label_im)
    if xs.size > 0:
        xmin = np.min(xs)
        pos_w = float(w) / num_classes
        cls_id = int(float(xmin) / pos_w)
        onehot_code[cls_id] = 1
    else:
        xmin = -1
        onehot_code[-1] = 1
        cls_id = num_classes
        
    return onehot_code, cls_id, xmin
        

In [ ]:
debug_display = False
sample_num = 100

x_offset_max = train_images[0].shape[1] - DETECTION_AREA_WIDTH
y_offset_max = train_images[0].shape[0] - DETECTION_AREA_HEIGHT
rng = np.random.RandomState(0)

# for each
# input_x, gnd_y
input_x = []
gnd_y = []
for i in range(sample_num):
    image_id = rng.randint(len(train_images))

    x0_, y0_ = rng.randint(x_offset_max), rng.randint(y_offset_max)
    x1_, y1_ = x0_ + DETECTION_AREA_WIDTH, y0_ + DETECTION_AREA_HEIGHT

    train_area = train_images[image_id][y0_:y1_, x0_:x1_, :]
    label_area = label_images[image_id][y0_:y1_, x0_:x1_]
    cls_code, cls_id, _ = get_label_code_h(label_area, LOCATION_NUM)
    input_x.append(np.rollaxis(preprocessor(train_area, mean_bgr),2))
    gnd_y.append(cls_id)

    if debug_display:
        im2 = train_images[image_id].copy()
        im2l = cv2.cvtColor(label_area, cv2.COLOR_GRAY2BGR)
        if cls_id < LOCATION_NUM:
            bx0_ = int(cls_id * float(DETECTION_AREA_WIDTH) / LOCATION_NUM)
            bx1_ = int((cls_id+1) * float(DETECTION_AREA_WIDTH) / LOCATION_NUM)
            im2l[:, bx0_:bx1_, 1] = 255
            print cls_id, cls_code
        else:
            im2l[..., 2] += 128
        
        im2[y0_:y1_, x0_:x1_, :] = \
            cv2.addWeighted(im2[y0_:y1_, x0_:x1_, :], 0.5, im2l, 0.5, 0)
        cv2.imshow("a", im2)
        cv2.waitKey()
        
if debug_display: 
    cv2.destroyAllWindows()

In [ ]:
a = np.random.permutation(100000)

In [ ]:
def get_sample():
    a = range(10)
    i = 0
    while True:
        yield a[i:i+2]
        i += 2
        if i>=9:
            i=0
    
    

In [ ]:
a = [ (i, iv)
     for i, iv in zip(range(10), get_sample()) ]


In [ ]:
np.random.shuffle(a)

In [ ]:
b.next()

# NET

In [ ]:
opts = {'image_width': DETECTION_AREA_WIDTH, 'image_height': DETECTION_AREA_HEIGHT}
ldet = LaneCloseDetectNet(opts)

In [ ]:
X = Variable(torch.from_numpy(np.ascontiguousarray(input_x)), requires_grad=False)

In [ ]:
Y = Variable(torch.LongTensor(gnd_y), requires_grad=False)

In [ ]:
pred = ldet(X)

In [ ]:
loss_fn = nn.NLLLoss()

In [ ]:
loss = loss_fn(pred, Y)

In [ ]:
LEARNING_RATE = 1e-4
optimiser = torch.optim.Adagrad(ldet.parameters(), lr=LEARNING_RATE)

In [ ]:
optimiser.zero_grad()

In [ ]:
loss.backward()

In [ ]:
optimiser.step()

In [ ]:
loss

In [ ]:
for i_ in range(10):
    pred = ldet(X)
    loss = loss_fn(pred, Y)
    print i_, loss
    
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()
    

# SAVE and LOAD

In [ ]:
fname = os.path.join('tmp-save-10')
torch.save(ldet.state_dict(), fname)                      
# with open(checkpoint_status_file, 'w') as f:                    
#     json.dump({'episode': episode,                              
# 309                                'running_loss': running_loss,                    
# 310                                'running_reward': running_reward,                
# 311                                'loss_history': loss_history,                    
# 312                                'reward_history': reward_history}, f, indent=2)  
# 313                 print "Save model to {}".format(fname)            

In [ ]:
ldet2 = LaneCloseDetectNet(opts)

In [ ]:
ldet2.load_state_dict(torch.load(fname))

In [ ]:

pred = ldet2(X)
loss = loss_fn(pred, Y)
print i_, loss

    

In [9]:
%run train_lane_detectpr.py

Total Batch-0, Episode-0, Batch-0, Loss 4.78446292877
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-1, Loss 4.31895923615
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-2, Loss 3.42922401428
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-3, Loss 3.61986994743
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-4, Loss 3.33856344223
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-5, Loss 3.84738588333
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-6, Loss 4.03854656219
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-7, Loss 4.38210868835
Model Saved to ../../RUNS/detector_snapshots/checkpoint-0
Total Batch-0, Episode-0, Batch-8, Loss 3.72667813301
Model Saved to ../../RUNS/detector_snapshots/check

KeyboardInterrupt: 

In [10]:
data = Data()
train_set = data.data_source('train', minibatch=20)
valid_set = data.data_source('valid', minibatch=20)

In [11]:
x,y,e,b = train_set.next()

In [7]:
with open('tmptmp', 'w') as f:
    json.dump({'a': a}, f)

TypeError: 4.6999998 is not JSON serializable

In [5]:
a = np.float32(4.7)

In [6]:
a

4.6999998